In [1]:
import Heston
import BlackScholes as bs
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import ImpliedDrift as iD
import time

from matplotlib import cm

In [2]:
# Read the dataset
spot = np.array(pd.read_csv("spot.csv").Spot).flatten()
index = 0; S0 = spot[index];

IV_df = pd.read_csv("hist_spx.csv")
moneyness = np.array([80.0,90.0,95.0,97.5,100.0,102.5,105.0,110.0,120.0])
maturities = np.array(IV_df['Exp Date']).flatten()
IV = np.array(IV_df.drop(columns = 'Exp Date'))/100.

K = S0 * moneyness / 100

param = np.array(pd.read_csv("Heston_data.csv"))

In [3]:
N = 64; L = 12;
r = iD.r(maturities, index); q = iD.q(maturities, index);
m = len(K); n =len(maturities);
options_type = K> S0

def f(x):
    rho,theta,kappa,eta,sigma_0 = x
    iv = np.zeros([n,m])
    for i in range(n):
        tau = maturities[i]
        precomp_term, a, b = Heston.precomputed_terms(r[i], q[i], tau, sigma_0, kappa, eta, theta, rho, L, N)
        P = Heston.cos_method_Heston_LF(precomp_term, a, b, tau, r[i], q[i], sigma_0, kappa, eta, theta, rho, 
                                       S0, K, N, options_type, L)
        iv[i,:5] = bs.BSImpliedVol(S0, K[:5], tau, r[i], q[i], P[:5], 0, toll = 1e-3)
        iv[i,5:] = bs.BSImpliedVol(S0, K[5:], tau, r[i], q[i], P[5:], 1, toll = 1e-3)
    return np.mean(abs(IV-iv)/IV)*100, iv

In [4]:
%%timeit
f(param[0])

1.7 s ± 46.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
t = time.time()
iv_list = list()
length = len(param); err = np.zeros(length);

for i in range(length):
    err[i], aux = f(param[i])
    iv_list.append(aux)
    if (i+1)%100 == 0:
        print(f"Iteration progress: {(i+1)/length*100: .2f}%")
    
print(f"\nElapsed time: {time.time()-t: .2f}s")

Iteration progress:  1.25%
Iteration progress:  2.50%
Iteration progress:  3.75%
Iteration progress:  5.00%
Iteration progress:  6.25%
Iteration progress:  7.50%
Iteration progress:  8.75%
Iteration progress:  10.00%
Iteration progress:  11.25%
Iteration progress:  12.50%
Iteration progress:  13.75%
Iteration progress:  15.00%
Iteration progress:  16.25%
Iteration progress:  17.50%
Iteration progress:  18.75%
Iteration progress:  20.00%
Iteration progress:  21.25%
Iteration progress:  22.50%
Iteration progress:  23.75%
Iteration progress:  25.00%
Iteration progress:  26.25%
Iteration progress:  27.50%
Iteration progress:  28.75%
Iteration progress:  30.00%
Iteration progress:  31.25%
Iteration progress:  32.50%
Iteration progress:  33.75%
Iteration progress:  35.00%
Iteration progress:  36.25%
Iteration progress:  37.50%
Iteration progress:  38.75%
Iteration progress:  40.00%
Iteration progress:  41.25%
Iteration progress:  42.50%
Iteration progress:  43.75%
Iteration progress:  45.00%

In [6]:
err.min()

3.223690256014374

In [7]:
err.max()

6.300560518271599

In [8]:
err.mean()

4.412205615422748

In [9]:
err.std()

0.4170241970581467

In [10]:
err.var()

0.17390918093199195

In [11]:
param[np.argmin(err)]

array([-0.73522207,  1.14729524,  3.15831869,  0.06290066,  0.03603295])

In [12]:
df = pd.DataFrame(err, columns = ["Error"])
df.to_csv("grid_error.csv", index = False)

In [13]:
iv = np.zeros([length, len(iv_list[0].flatten())])

for i in range(length):
    iv[i] = iv_list[i].flatten()

In [14]:
db = pd.DataFrame(iv)
db.to_csv("grid_iv.csv", index = False)